In [2]:
!pip install requests beautifulsoup4 pandas selenium webdriver-manager
!apt-get update
!apt-get install -y wget unzip libglib2.0-0 libnss3 libgconf-2-4 libfontconfig1
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb || apt-get -f install -y
!rm google-chrome-stable_current_amd64.deb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.9/492.9 kB 20.6 MB/s eta 0:00:00
Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,538 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu j

In [13]:
import requests
from bs4 import BeautifulSoup
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import re

# URL cơ sở của trang danh mục thuốc dị ứng
BASE_URL = "https://nhathuoclongchau.com.vn"
CATEGORY_URL = f"{BASE_URL}/thuoc/thuoc-dieu-tri-ung-thu"

# Tiêu đề HTTP để giả lập trình duyệt
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

# Hàm làm sạch văn bản để đảm bảo khoảng cách giữa các từ
def clean_text(text):
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Cài đặt Chrome và ChromeDriver cho Google Colab
def setup_chrome_driver():
    # Cấu hình ChromeDriver
    options = Options()
    options.add_argument('--headless')  # Chạy ẩn danh
    options.add_argument('--disable-gpu')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.binary_location = "/usr/bin/google-chrome"  # Đường dẫn đến Chrome binary trong Colab

    # Tải và cài đặt ChromeDriver tự động
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)
    return driver

# Hàm lấy danh sách liên kết sản phẩm bằng Selenium
def get_product_links(category_url):
    driver = setup_chrome_driver()
    driver.get(category_url)
    product_links = set()  # Sử dụng set để tránh trùng lặp
    wait = WebDriverWait(driver, 10)  # Chờ tối đa 10 giây cho các phần tử tải

    while True:
        # Lấy HTML hiện tại sau khi trang tải
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        links = soup.find_all('a', href=True)

        # Thu thập các liên kết sản phẩm
        for link in links:
            href = link['href']
            if href.startswith('/thuoc/') and href.endswith('.html'):
                full_url = f"{BASE_URL}{href}"
                product_links.add(full_url)

        # Kiểm tra nút "Xem thêm" với XPath mới
        try:
            load_more_button = wait.until(EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div[1]/div[2]/div[3]/div/div[4]/div/section/div[2]/button")))
            if load_more_button.is_displayed() and load_more_button.is_enabled():
                load_more_button.click()
                print(f"Đã nhấn 'Xem thêm'. Tổng liên kết: {len(product_links)}")
                time.sleep(2)
            else:
                print("Nút 'Xem thêm' không khả dụng hoặc đã đến cuối danh sách.")
                break
        except Exception as e:
            print("Đã đến cuối danh sách hoặc lỗi:", e)
            break

    driver.quit()
    return list(product_links)

# Hàm trích xuất thông tin từ trang sản phẩm
def scrape_product_info(product_url):
    response = requests.get(product_url, headers=HEADERS)
    if response.status_code != 200:
        print(f"Không thể truy cập {product_url}. Mã trạng thái: {response.status_code}")
        return None

    soup = BeautifulSoup(response.text, 'html.parser')

    # Tạo từ điển lưu thông tin sản phẩm
    product_info = {
        "url": product_url,
        "content_list": "",
        "content_container": "",
        "product_content": "",
        "heading": "",
        "inner_div": "",
        "ingredient": "",
        "usage": "",
        "dosage": "",
        "adverse_effect": "",
        "careful": "",
        "preservation": ""
    }

    # Trích xuất thông tin từ các thẻ HTML
    content_list = soup.find('table', class_='content-list')
    product_info["content_list"] = clean_text(content_list.get_text(separator=' ')) if content_list else ""

    content_container = soup.find('div', class_='content-container')
    product_info["content_container"] = clean_text(content_container.get_text(separator=' ')) if content_container else ""

    product_content = soup.find('div', {'data-lcpr': 'prr-id-product-detail-product-content'})
    product_info["product_content"] = clean_text(product_content.get_text(separator=' ')) if product_content else ""

    heading = soup.find('p', class_='text-heading2 lc-tit font-semibold md:basis-[669px]')
    product_info["heading"] = clean_text(heading.get_text(separator=' ')) if heading else ""

    inner_div = soup.find('div', class_='inner')
    product_info["inner_div"] = clean_text(inner_div.get_text(separator=' ')) if inner_div else ""

    product_info["ingredient"] = clean_text(soup.find('div', class_='ingredient', id='detail-content-0').get_text(separator=' ')) if soup.find('div', class_='ingredient', id='detail-content-0') else ""
    product_info["usage"] = clean_text(soup.find('div', class_='usage', id='detail-content-1').get_text(separator=' ')) if soup.find('div', class_='usage', id='detail-content-1') else ""
    product_info["dosage"] = clean_text(soup.find('div', class_='dosage', id='detail-content-2').get_text(separator=' ')) if soup.find('div', class_='dosage', id='detail-content-2') else ""
    product_info["adverse_effect"] = clean_text(soup.find('div', class_='adverseEffect', id='detail-content-3').get_text(separator=' ')) if soup.find('div', class_='adverseEffect', id='detail-content-3') else ""
    product_info["careful"] = clean_text(soup.find('div', class_='careful', id='detail-content-4').get_text(separator=' ')) if soup.find('div', class_='careful', id='detail-content-4') else ""
    product_info["preservation"] = clean_text(soup.find('div', class_='preservation', id='detail-content-5').get_text(separator=' ')) if soup.find('div', class_='preservation', id='detail-content-5') else ""

    return product_info

# Hàm chính để chạy công cụ
def main():
    print("Bắt đầu cào dữ liệu từ", CATEGORY_URL)
    product_links = get_product_links(CATEGORY_URL)
    print(f"Tìm thấy {len(product_links)} sản phẩm.")

    all_products = []
    for i, link in enumerate(product_links, 1):
        print(f"Đang xử lý sản phẩm {i}/{len(product_links)}: {link}")
        product_info = scrape_product_info(link)
        if product_info:
            all_products.append(product_info)
        time.sleep(0.5)  # Độ trễ để tránh bị chặn

    # Lưu dữ liệu vào file CSV
    df = pd.DataFrame(all_products)
    df.to_csv("thuoc-dieu-tri-ung-thu.csv", index=False, encoding='utf-8-sig')
    print("Đã lưu dữ liệu vào file")

if __name__ == "__main__":
    main()

Bắt đầu cào dữ liệu từ https://nhathuoclongchau.com.vn/thuoc/thuoc-dieu-tri-ung-thu
Đã nhấn 'Xem thêm'. Tổng liên kết: 23
Đã nhấn 'Xem thêm'. Tổng liên kết: 31
Đã nhấn 'Xem thêm'. Tổng liên kết: 39
Đã nhấn 'Xem thêm'. Tổng liên kết: 47
Đã nhấn 'Xem thêm'. Tổng liên kết: 55
Đã nhấn 'Xem thêm'. Tổng liên kết: 63
Đã nhấn 'Xem thêm'. Tổng liên kết: 71
Đã nhấn 'Xem thêm'. Tổng liên kết: 79
Đã nhấn 'Xem thêm'. Tổng liên kết: 87
Đã nhấn 'Xem thêm'. Tổng liên kết: 95
Đã nhấn 'Xem thêm'. Tổng liên kết: 103
Đã nhấn 'Xem thêm'. Tổng liên kết: 111
Đã nhấn 'Xem thêm'. Tổng liên kết: 119
Đã nhấn 'Xem thêm'. Tổng liên kết: 127
Đã nhấn 'Xem thêm'. Tổng liên kết: 135
Đã nhấn 'Xem thêm'. Tổng liên kết: 143
Đã đến cuối danh sách hoặc lỗi: Message: 
Stacktrace:
#0 0x5d02e05f8ffa <unknown>
#1 0x5d02e00b7970 <unknown>
#2 0x5d02e0109385 <unknown>
#3 0x5d02e01095b1 <unknown>
#4 0x5d02e01583c4 <unknown>
#5 0x5d02e012f2bd <unknown>
#6 0x5d02e015570c <unknown>
#7 0x5d02e012f063 <unknown>
#8 0x5d02e00fb328 <unkno

KeyboardInterrupt: 

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import re

base_url = 'https://www.vinmec.com'

# Tiêu đề HTTP để giả lập trình duyệt
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

# Hàm làm sạch văn bản để đảm bảo khoảng cách giữa các từ
def clean_text(text):
    text = re.sub(r'\s+', ' ', text).strip()
    return text.split('Xem thêm')[0]

# Cài đặt Chrome và ChromeDriver
def setup_chrome_driver():
    options = Options()
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)
    return driver


def get_all_disease_links(driver):
    disease_links = set()
    wait = WebDriverWait(driver, 10)

    for letter in 'abcdefghijklmnopqrstuvwxyz':
        print(f'Đang thu thập danh sách bệnh bắt đầu với chữ: {letter}')
        url = f'{base_url}/vie/tra-cuu-benh/{letter}/'
        driver.get(url)

        while True:
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            disease_list = soup.find('ul', class_='list_result_AZ flex')
            if disease_list:
                for li in disease_list.find_all('li'):
                    a_tag = li.find('a')
                    if a_tag:
                        disease_links.add((a_tag.text.strip(), base_url + a_tag['href']))

            try:
                next_page = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'next_page')))
                next_page.click()
                time.sleep(1)
            except Exception:
                break

    return list(disease_links)


def get_disease_info(disease_url):
    response = requests.get(disease_url, headers=HEADERS)
    if response.status_code != 200:
        return None

    soup = BeautifulSoup(response.content, 'html.parser')
    content_divs = soup.find_all('div', class_='item_detial_sick')

    disease_data = {
        "Tổng quan": "",
        "Nguyên nhân": "",
        "Triệu chứng": "",
        "Đối tượng nguy cơ": "",
        "Phòng ngừa": "",
        "Chẩn đoán": "",
        "Điều trị": ""
    }

    for i, (key) in enumerate(disease_data.keys()):
        if i < len(content_divs):
            disease_data[key] = clean_text(content_divs[i].get_text(separator=' ', strip=True))

    return disease_data


def main():
    csv_filename = 'disease_data.csv'
    total_collected = 0
    driver = setup_chrome_driver()

    print('Đang thu thập danh sách tất cả các bệnh...')
    disease_links = get_all_disease_links(driver)
    total_diseases = len(disease_links)
    print(f'Tìm thấy {total_diseases} bệnh cần thu thập dữ liệu.')

    data_list = []
    for index, (name, link) in enumerate(disease_links, start=1):
        print(f'[{index}/{total_diseases}] Đang thu thập dữ liệu từ: {link}')
        disease_info = get_disease_info(link)
        if disease_info:
            data_list.append([name, link] + list(disease_info.values()))
            total_collected += 1
        time.sleep(0.5)

    df = pd.DataFrame(data_list, columns=['Tên bệnh', 'URL', 'Tổng quan', 'Nguyên nhân', 'Triệu chứng', 'Đối tượng nguy cơ', 'Phòng ngừa', 'Chẩn đoán', 'Điều trị'])
    df.to_csv(csv_filename, index=False, encoding='utf-8-sig')

    driver.quit()
    print(f'Hoàn thành thu thập! Tổng số bệnh thu thập thành công: {total_collected}/{total_diseases}')

if __name__ == '__main__':
    main()

Đang thu thập danh sách tất cả các bệnh...
Đang thu thập danh sách bệnh bắt đầu với chữ: a
Đang thu thập danh sách bệnh bắt đầu với chữ: b
Đang thu thập danh sách bệnh bắt đầu với chữ: c
Đang thu thập danh sách bệnh bắt đầu với chữ: d
Đang thu thập danh sách bệnh bắt đầu với chữ: e
Đang thu thập danh sách bệnh bắt đầu với chữ: f
Đang thu thập danh sách bệnh bắt đầu với chữ: g
Đang thu thập danh sách bệnh bắt đầu với chữ: h
Đang thu thập danh sách bệnh bắt đầu với chữ: i
Đang thu thập danh sách bệnh bắt đầu với chữ: j
Đang thu thập danh sách bệnh bắt đầu với chữ: k
Đang thu thập danh sách bệnh bắt đầu với chữ: l
Đang thu thập danh sách bệnh bắt đầu với chữ: m
Đang thu thập danh sách bệnh bắt đầu với chữ: n
Đang thu thập danh sách bệnh bắt đầu với chữ: o
Đang thu thập danh sách bệnh bắt đầu với chữ: p
Đang thu thập danh sách bệnh bắt đầu với chữ: q
Đang thu thập danh sách bệnh bắt đầu với chữ: r
Đang thu thập danh sách bệnh bắt đầu với chữ: s
Đang thu thập danh sách bệnh bắt đầu với chữ: